In [1]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns

# 결과 저장 경로 생성
output_dir = "../results/figures/"
os.makedirs(output_dir, exist_ok=True)

# CSV 파일들을 읽어 하나의 DataFrame으로 병합하는 함수
def read_all_csv_to_df(directory_path):
    # 디렉토리 내 모든 파일 가져오기
    files = os.listdir(directory_path)
    # 확장자가 .csv인 파일만 필터링
    csv_files = [file for file in files if file.endswith('.csv')]
    
    # CSV 파일들 읽어서 병합
    dataframes = []
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path)
        dataframes.append(df)
    
    # 모든 DataFrame 병합 (행 방향으로)
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# ../data/raw 경로에서 CSV 읽기
directory_path = "../data/raw/"
df = read_all_csv_to_df(directory_path)

In [2]:
df.columns = df.columns.str.strip()

In [5]:
columns_dict = {
    "포트 및 트래픽량 관련": [
        "Flow Duration",  # 주요 칼럼
        "Total Fwd Packets",  # 주요 칼럼
        "Total Backward Packets",  # 주요 칼럼
        "Flow Bytes/s",  # 주요 칼럼
        "Flow Packets/s",  # 주요 칼럼
        "Destination Port"
    ],
    "패킷 길이 관련": [
        "Fwd Packet Length Max",  # 주요 칼럼
        "Fwd Packet Length Mean",  # 주요 칼럼
        "Bwd Packet Length Max",  # 주요 칼럼
        "Bwd Packet Length Mean",  # 주요 칼럼
        "Total Length of Fwd Packets",
        "Total Length of Bwd Packets",
        "Fwd Packet Length Min",
        "Fwd Packet Length Std",
        "Bwd Packet Length Min",
        "Bwd Packet Length Std",
        "Min Packet Length",
        "Max Packet Length",
        "Packet Length Mean",
        "Packet Length Std",
        "Packet Length Variance"
    ],
    "플래그 및 헤더 관련": [
        "PSH Flag Count",  # 주요 칼럼
        "URG Flag Count",  # 주요 칼럼
        "SYN Flag Count",  # 주요 칼럼
        "FIN Flag Count",  # 주요 칼럼
        "RST Flag Count",  # 주요 칼럼
        "Fwd PSH Flags",
        "Bwd PSH Flags",
        "Fwd URG Flags",
        "Bwd URG Flags",
        "Fwd Header Length",
        "Bwd Header Length",
        "Fwd Header Length.1",
        "ACK Flag Count",
        "CWE Flag Count",
        "ECE Flag Count"
    ],
    "속도 및 비율 관련": [
        "Flow Bytes/s",  
        "Flow Packets/s",  
        "Down/Up Ratio",
        "Average Packet Size",
        "Fwd Avg Bytes/Bulk",
        "Fwd Avg Packets/Bulk",
        "Fwd Avg Bulk Rate",
        "Bwd Avg Bytes/Bulk",
        "Bwd Avg Packets/Bulk",
        "Bwd Avg Bulk Rate"
    ],
    "세그먼트 및 하위 플로우 관련": [
        "Avg Fwd Segment Size",
        "Avg Bwd Segment Size",
        "Subflow Fwd Packets",
        "Subflow Fwd Bytes",
        "Subflow Bwd Packets",
        "Subflow Bwd Bytes"
    ],
    "시간 관련": [
        "Flow IAT Mean",  # 주요 칼럼
        "Flow IAT Std",  # 주요 칼럼
        "Flow IAT Max",
        "Flow IAT Min",
        "Fwd IAT Total",
        "Fwd IAT Mean",
        "Fwd IAT Std",
        "Fwd IAT Max",
        "Fwd IAT Min",
        "Bwd IAT Total",
        "Bwd IAT Mean",
        "Bwd IAT Std",
        "Bwd IAT Max",
        "Bwd IAT Min"
    ],
    "윈도우 크기 및 기타": [
        "Init_Win_bytes_forward",
        "Init_Win_bytes_backward",
        "act_data_pkt_fwd",
        "min_seg_size_forward"
    ],
    "활동 및 휴면 시간 관련": [
        "Active Mean",
        "Active Std",
        "Active Max",
        "Active Min",
        "Idle Mean",
        "Idle Std",
        "Idle Max",
        "Idle Min"
    ],
    "레이블": [
        "Label"  # 주요 칼럼
    ]
}


In [3]:
# 한글 폰트 경로 설정
font_path = r"C:\Windows\Fonts\batang.ttc" 
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 한글 폰트 적용 확인
print(f"설정된 폰트: {font_name}")

설정된 폰트: Batang


In [ ]:
def column_visualization(df, columns_dict, output_dir):

    # 각 카테고리별로 반복
    for category, columns in columns_dict.items():
        for column in columns:
            if column not in df.columns:
                print(f"칼럼 {column}이(가) 데이터프레임에 없습니다. 건너뜁니다.")
                continue
            
            safe_column_name = re.sub(r'[<>:"/\\|?*]', '_', column) 
            
            # 'Label' 칼럼인 경우 파이 차트 생성
            if column == 'Label':
                plt.figure(figsize=(8, 6))
                
                label_counts = df[column].value_counts()
                labels = label_counts.index
                
                # 상위 4개 항목과 나머지 항목을 "기타"로 묶기
                top_labels = labels[:4]  # 상위 4개 레이블
                top_counts = label_counts.head(4)  # 상위 4개 항목의 개수
                
                # 나머지 항목을 '기타'로 묶음
                other_count = label_counts[4:].sum()
                other_label = '기타'
                
                # 새로운 레이블과 개수 배열 생성
                final_labels = list(top_labels) + [other_label]
                final_counts = list(top_counts) + [other_count]
                
                # 파이 차트 그리기
                wedges, texts, autotexts = plt.pie(
                    final_counts, 
                    labels=final_labels, 
                    autopct='%1.1f%%',  # 비율 표시
                    startangle=90,  # 시작 각도 설정
                    wedgeprops={'edgecolor': 'black', 'linewidth': 1.5},  # 조각 테두리 설정
                    textprops={'fontsize': 10},  # 텍스트 크기 설정
                    pctdistance=0.85,  # 비율 텍스트 위치 설정
                    labeldistance=1.1,  # 레이블 위치 설정 (기본값보다 바깥쪽으로 설정)
                    colors=['skyblue', 'lightcoral', 'lightgreen', 'orange', 'gray']  # 색깔 설정
                )

                # 레이블 텍스트가 겹치지 않도록 회전
                for t in texts:
                    t.set_rotation(-45)  # 각도를 -45도로 회전시켜 겹침 방지

                # 파이 차트 제목
                plt.title(f"- {column} 파이 차트")
                
                # 범례 추가
                plt.legend(wedges, final_labels, title="Labels", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

                # 차트 저장
                bar_chart_path = os.path.join(output_dir, f"{category}_{safe_column_name}_pie_chart.png")
                plt.savefig(bar_chart_path, bbox_inches='tight')  # bbox_inches='tight'로 여백을 최소화
                plt.close()
                
                plt.figure(figsize=(8, 6))
    
                # 레이블 빈도 계산
                label_counts = df[column].value_counts()
                
                # 빈도에 대한 바 차트 그리기
                sns.barplot(y=label_counts.index, x=label_counts.values, palette='Set2')
                
                # 차트 제목, x축, y축 레이블 설정
                plt.title(f"{category} - {column} 레이블 빈도")
                plt.ylabel(column)  # 이제 y축이 레이블
                plt.xlabel('빈도수')  # 이제 x축이 빈도수
                
                # 빈도 수 숫자 표시
                for i, count in enumerate(label_counts.values):
                    plt.text(count + 1, i, str(count), va='center', fontsize=10)  # 각 바 옆에 빈도 표시
                
                # y축 레이블을 90도 회전하여 표시
                plt.yticks(rotation=0)

                # 차트 저장
                bar_chart_path = os.path.join(output_dir, f"{category}_{safe_column_name}_bar_chart.png")
                plt.savefig(bar_chart_path, bbox_inches='tight')  # bbox_inches='tight'로 여백을 최소화
                plt.close()
            
            else:
                # 히스토그램 생성
                plt.figure(figsize=(8, 6))
                sns.histplot(df[column], kde=True, color='skyblue', bins=30)
                mean_value = df[column].mean()
                median_value = df[column].median()
                plt.axvline(mean_value, color='red', linestyle='--', label=f'Mean: {mean_value:.2f}')
                plt.axvline(median_value, color='green', linestyle='-', label=f'Median: {median_value:.2f}')
                plt.legend()
                plt.title(f"{category} - {column} 히스토그램")
                plt.xlabel(column)
                plt.ylabel('Frequency')

                # 히스토그램 저장
                hist_path = os.path.join(output_dir, f"{category}_{safe_column_name}_histogram.png")
                plt.savefig(hist_path)
                plt.close()

                # 박스플롯 생성
                plt.figure(figsize=(8, 6))
                sns.boxplot(x=df[column], color='lightblue')
                plt.title(f"{category} - {column} 박스플롯")
                plt.xlabel(column)

                # 박스플롯 저장
                boxplot_path = os.path.join(output_dir, f"{category}_{safe_column_name}_boxplot.png")
                plt.savefig(boxplot_path)
                plt.close()


# 함수 호출
column_visualization(df, columns_dict, "../results/figures")

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

# 라벨 인코딩
label_mapping = {
    'BENIGN': 0, 'FTP-Patator': 1, 'SSH-Patator': 2, 'DoS slowloris': 3,
    'DoS Slowhttptest': 4, 'DoS Hulk': 5, 'DoS GoldenEye': 6, 'Heartbleed': 7,
    'Web Attack � Brute Force': 8, 'Web Attack � XSS': 9,
    'Web Attack � Sql Injection': 10, 'Infiltration': 11, 'Bot': 12,
    'PortScan': 13, 'DDoS': 14
}

df['Label'] = df['Label'].map(label_mapping)

# 각 칼럼에서 음수인 값 개수 확인
negative_column_counts = df.lt(0).sum()

# 음수가 있는 칼럼만 필터링
columns_with_negative_values = negative_column_counts[negative_column_counts > 0]

# 결과를 DataFrame으로 변환하여 칼럼명 설정
negative_columns_df = columns_with_negative_values.reset_index()
negative_columns_df.columns = ['Negative columns name', 'Negative Count']

# 음수가 포함된 데이터 수 (각 row에서 음수가 포함된 값의 개수)
negative_values_count = (df < 0).sum().sum()

# 전체 데이터의 개수
total_data_count = df.size

# 상위 n개의 칼럼 분리
top_n = 2
top_columns = negative_columns_df.nlargest(top_n, 'Negative Count')
other_columns = negative_columns_df.drop(top_columns.index)

# 상위 칼럼 시각화
plt.figure(figsize=(8, 4))
sns.barplot(x=top_columns['Negative columns name'], y=top_columns['Negative Count'])
plt.title(f"음수 값 개수 (상위 {top_n} 칼럼)")
plt.ylabel("음수 개수")
plt.xlabel("칼럼 이름")
negative_columns_top2 = os.path.join(output_dir, f"negative_columns_top2.png")
plt.savefig(negative_columns_top2)
plt.close()

# 나머지 칼럼 시각화
plt.figure(figsize=(12, 6))
sns.barplot(x=other_columns['Negative columns name'], y=other_columns['Negative Count'])
plt.xticks(rotation=45, ha="right")
plt.title("음수 값 개수 (나머지 칼럼)")
plt.ylabel("음수 개수")
plt.xlabel("칼럼 이름")
plt.tight_layout()
negative_columns_rest = os.path.join(output_dir, f"negative_columns_rest.png")
plt.savefig(negative_columns_rest)
plt.close()


# 음수 데이터 비율
labels = ['Negative Values', 'Non-Negative Values']
sizes = [negative_values_count, total_data_count - negative_values_count]
colors = ['#ff9999', '#66b3ff']
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
plt.title("전체 데이터에서 음수 데이터 비율")
negative_data_ratio = os.path.join(output_dir, f"negative_data_ratio.png")
plt.savefig(negative_data_ratio)
plt.close()

In [29]:
import networkx as nx

# 상관계수 계산
correlation_matrix = df.corr()

# 상관계수 절대값 기준 상위 k개의 값 선택
k = 30
corr_unstacked = correlation_matrix.abs().unstack()
corr_unstacked = corr_unstacked[corr_unstacked < 1].sort_values(ascending=False)  # 1 제외
top_k_pairs = corr_unstacked.head(k).index

# 상위 k개의 칼럼 추출
selected_columns = list(set([i for pair in top_k_pairs for i in pair]))

# 선택된 칼럼의 상관행렬 생성
selected_corr_matrix = correlation_matrix.loc[selected_columns, selected_columns]

# 히트맵 시각화
plt.figure(figsize=(12, 12))
sns.heatmap(selected_corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title(f"Top {k} Correlated Columns Heatmap")
plt.savefig( os.path.join(output_dir, f"corr.png"))
plt.close()

# 특정 상관계수 이상의 쌍만 선택
threshold = 0.9
edges = correlation_matrix.stack().reset_index()
edges.columns = ['var1', 'var2', 'correlation']
edges = edges[(edges['var1'] != edges['var2']) & (edges['correlation'].abs() >= threshold)]

# 네트워크 생성
G = nx.Graph()
for _, row in edges.iterrows():
    G.add_edge(row['var1'], row['var2'], weight=row['correlation'])

# 네트워크 시각화
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42)  # 레이아웃 설정
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="skyblue")
nx.draw_networkx_edges(G, pos, edge_color="gray", alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=8, font_color="black")
plt.title("Correlation Network (Threshold ≥ 0.9)")
plt.savefig( os.path.join(output_dir, f"corr_network.png"))
plt.close()


In [4]:
columns_to_drop = []

# 단일 값으로 이루어진 칼럼
columns_to_drop.extend(['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 
                        'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 
                        'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'])

# 중복된 칼럼
columns_to_drop.extend(['Subflow Fwd Packets', 'Subflow Bwd Packets', 'Avg Fwd Segment Size', 
                        'Avg Bwd Segment Size', 'SYN Flag Count', 'CWE Flag Count', 'Fwd Header Length.1'])

# 칼럼 제거
df_cleaned = df.drop(columns=columns_to_drop)

In [11]:
# 원래 칼럼 수와 제거된 후 칼럼 수
original_columns = len(df.columns) - 1
remaining_columns = len(df_cleaned.columns) - 1

# 칼럼 제거된 개수
removed_columns = original_columns - remaining_columns

# 데이터 준비
data = {
    'Status': ['Before Removal', 'After Removal'],
    'Count': [original_columns, remaining_columns]
}

# DataFrame 생성
df2 = pd.DataFrame(data)

# 시각화
plt.figure(figsize=(8, 6))
sns.barplot(x='Status', y='Count', data=df2)

# 칼럼 수와 제거된 개수 표시
plt.title(f"칼럼 제거 전후 (삭제된 칼럼: {removed_columns}개)", fontsize=16)
plt.ylabel("칼럼 수", fontsize=12)
plt.xlabel("상태", fontsize=12)
plt.savefig( os.path.join(output_dir, f"col_drop.png"))
plt.close()
